In [1]:
import numpy as np
import datetime
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
df = pd.read_excel('./Attachment 1.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16963 entries, 0 to 16962
Data columns (total 4 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   日期(年/月/日) (Date Y/M/D)                       16963 non-null  datetime64[ns]
 1   发货城市 (Delivering city)                       16963 non-null  object        
 2   收货城市 (Receiving city)                        16963 non-null  object        
 3   快递运输数量(件) (Express delivery quantity (PCS))  16963 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 530.2+ KB


In [4]:
df.columns = ["日期","发货城市","收货城市","快递运输数量"]

In [5]:
df

,日期,发货城市,收货城市,快递运输数量
0,2018-04-19,A,O,45
1,2018-04-19,S,R,51
2,2018-04-19,S,Q,42
3,2018-04-19,S,L,56
4,2018-04-19,S,I,42
...,...,...,...,...
16958,2019-04-17,J,H,47
16959,2019-04-17,I,J,220
16960,2019-04-17,I,E,70
16961,2019-04-17,K,H,241


In [6]:
df["线路"] = df["发货城市"] +"->"+ df["收货城市"]

In [7]:
df

,日期,发货城市,收货城市,快递运输数量,线路
0,2018-04-19,A,O,45,A->O
1,2018-04-19,S,R,51,S->R
2,2018-04-19,S,Q,42,S->Q
3,2018-04-19,S,L,56,S->L
4,2018-04-19,S,I,42,S->I
...,...,...,...,...,...
16958,2019-04-17,J,H,47,J->H
16959,2019-04-17,I,J,220,I->J
16960,2019-04-17,I,E,70,I->E
16961,2019-04-17,K,H,241,K->H


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16963 entries, 0 to 16962
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   日期      16963 non-null  datetime64[ns]
 1   发货城市    16963 non-null  object        
 2   收货城市    16963 non-null  object        
 3   快递运输数量  16963 non-null  int64         
 4   线路      16963 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 662.7+ KB


In [9]:
from keras.activations import softplus

In [10]:
data = pd.read_excel('./Attachment 1.xlsx').values
lines = np.unique(np.array([i[0]+i[1]for i in data[:,[1,2]]]))
lines = np.array([[i[0],i[1]] for i in lines])

all_pre = []
for i in range(lines.shape[0]):
    y = data[np.logical_and(data[:,1]==lines[i,0],data[:,2]==lines[i,1])][:,-1].astype(np.float64)
    # 将数据划分为训练集和测试集
    train_size = int(len(y) * 0.8)
    train, test = y[:train_size], y[train_size:]

    # 数据归一化
    scaler = MinMaxScaler()
    train = scaler.fit_transform(train.reshape(-1, 1))
    test = scaler.transform(test.reshape(-1, 1))

    # 创建数据生成器
    def create_dataset(data, look_back=1):
        X, Y = [], []
        for i in range(len(data) - look_back):
            X.append(data[i:(i + look_back), 0])
            Y.append(data[i + look_back, 0])
        return np.array(X), np.array(Y)

    look_back = 7
    X_train, y_train = create_dataset(train, look_back)
    X_test, y_test = create_dataset(test, look_back)

    # 创建BP神经网络模型
    #model = Sequential()
    #model.add(Dense(64, input_dim=look_back, activation='relu'))
    #model.add(Dense(32, activation='relu'))
    #model.add(Dense(1))
    #model.compile(loss='mean_squared_error', optimizer='adam')
    # 创建BP神经网络模型
    model = Sequential()
    model.add(Dense(64, input_dim=look_back, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation=softplus))
    model.compile(loss='mean_squared_error', optimizer='adam')

    # 训练模型
    model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=1)

    # 预测
    train_predict = model.predict(X_train)
    test_predict = model.predict(X_test)

    # 将预测结果转换回原始尺度
    train_predict = scaler.inverse_transform(train_predict)
    y_train = scaler.inverse_transform(y_train.reshape(-1, 1))
    test_predict = scaler.inverse_transform(test_predict)
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

    # 计算预测准确性，例如使用均方误差（MSE）作为评估指标
    train_mse = np.mean((train_predict - y_train) ** 2)
    test_mse = np.mean((test_predict - y_test) ** 2)

    # 计算预测日期与最后一个训练日期之间的天数
    last_train_date = datetime.date(2019, 4, 17)
    start_pred_date = datetime.date(2019, 4, 18)
    end_pred_date = datetime.date(2019, 4, 20)
    days_to_predict = (end_pred_date-start_pred_date).days

    # 使用训练数据的最后一部分来开始预测
    input_data = train[-look_back:]

    predictions = []

    # 预测每一天的货量
    for i in range(days_to_predict):
        input_data_reshaped = input_data.reshape(1, look_back)
        pred = model.predict(input_data_reshaped)
        predictions.append(pred[0, 0])

        # 更新输入数据，用预测值替换最早的值
        input_data = np.roll(input_data, -1)
        input_data[-1] = pred

    # 将预测值转换回原始尺度
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

    tdays = []
    # 打印预测结果
    for i, pred in enumerate(predictions, start=1):
        pred_date = start_pred_date + datetime.timedelta(days=i - 1)
        tdays += [pred[0]]
    all_pre+=[tdays]
#    print(tdays)

Epoch 1/100
129/129 [==============================] - 1s 686us/step - loss: 0.0315
Epoch 2/100
129/129 [==============================] - 0s 670us/step - loss: 0.0209
Epoch 3/100
129/129 [==============================] - 0s 640us/step - loss: 0.0174
Epoch 4/100
129/129 [==============================] - 0s 678us/step - loss: 0.0143
Epoch 5/100
129/129 [==============================] - 0s 639us/step - loss: 0.0130
Epoch 6/100
129/129 [==============================] - 0s 701us/step - loss: 0.0127
Epoch 7/100
129/129 [==============================] - 0s 647us/step - loss: 0.0123
Epoch 8/100
129/129 [==============================] - 0s 631us/step - loss: 0.0125
Epoch 9/100
129/129 [==============================] - 0s 629us/step - loss: 0.0111
Epoch 10/100
129/129 [==============================] - 0s 631us/step - loss: 0.0119
Epoch 11/100
129/129 [==============================] - 0s 623us/step - loss: 0.0114
Epoch 12/100
129/129 [==============================] - 0s 623us/step - lo

In [11]:
all_pre = np.array(all_pre)

In [12]:
all_pre

array([[105.51654  , 105.996445 ],
       [ 46.918247 ,  51.223557 ],
       [276.25717  , 268.36914  ],
       [ 47.694744 ,  48.349617 ],
       [ 20.573166 ,   5.321654 ],
       [ 40.62287  ,  39.24163  ],
       [ 37.816948 ,  37.421    ],
       [ 53.99958  ,  54.64651  ],
       [ 71.35763  ,  66.3093   ],
       [ 37.959724 ,  23.303293 ],
       [ 28.499443 ,  21.409212 ],
       [183.50183  , 164.90323  ],
       [504.23907  , 562.2121   ],
       [  5.4002166,  21.522207 ],
       [255.49509  , 253.04382  ],
       [ 61.31067  ,  62.82035  ],
       [ 57.606556 ,  55.927666 ],
       [541.10846  , 567.0313   ],
       [ 43.866024 ,  41.448044 ],
       [ 51.078114 ,  78.91957  ],
       [ 54.4802   ,  54.353867 ],
       [ 49.22084  ,  48.874184 ],
       [113.28928  , 131.54543  ],
       [273.54816  , 264.32086  ],
       [ 65.41008  ,  67.300674 ],
       [ 42.87245  ,  42.600433 ],
       [309.19592  , 291.18988  ],
       [109.59094  , 175.79536  ],
       [112.79752  ,

In [13]:
pd.DataFrame(np.c_["1",lines,all_pre],columns=["起点","终点","2023年4月18日","2023年4月19日"]).to_excel("q2results.xlsx")

In [14]:
df_result = pd.read_excel('./q2results.xlsx')

In [15]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  91 non-null     int64  
 1   起点          91 non-null     object 
 2   终点          91 non-null     object 
 3   2023年4月18日  91 non-null     float64
 4   2023年4月19日  91 non-null     float64
dtypes: float64(2), int64(1), object(2)
memory usage: 3.7+ KB


In [16]:
df_result.drop(['Unnamed: 0'],axis=1,inplace=True)

In [17]:
df_result

,起点,终点,2023年4月18日,2023年4月19日
0,A,O,105.516540,105.996445
1,A,Q,46.918247,51.223557
2,B,G,276.257170,268.369140
3,C,M,47.694744,48.349617
4,C,N,20.573166,5.321654
...,...,...,...,...
86,X,W,543.032200,445.643460
87,X,Y,142.219760,200.273220
88,Y,L,74.219340,42.863754
89,Y,W,257.280520,252.090740


In [18]:
# 定义路线列表
routes = [['M', 'U'], ['Q', 'V'], ['K', 'L'], ['G', 'V'], ['V', 'G'], ['A', 'Q'], ['D', 'A'], ['L', 'K']]

# 创建新的 DataFrame
new_df = pd.DataFrame(columns=df_result.columns)

# 遍历原始 DataFrame 中的每一行数据
for i, row in df_result.iterrows():
    # 获取起点和终点城市的名称
    start, end = row["起点"], row["终点"]

    # 判断起点和终点城市是否在路线列表中
    if [start, end] in routes:
        # 将该行数据添加到新的 DataFrame 中
        new_df = new_df.append(row)

E:\Users\panyue\AppData\Local\Temp\ipykernel_7208\2718128805.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row)
E:\Users\panyue\AppData\Local\Temp\ipykernel_7208\2718128805.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row)
E:\Users\panyue\AppData\Local\Temp\ipykernel_7208\2718128805.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row)
E:\Users\panyue\AppData\Local\Temp\ipykernel_7208\2718128805.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row)
E:\Users\panyue\AppData\Local\Temp\ipykernel_7208\2718128805.py:15: FutureWa

In [19]:
new_df

,起点,终点,2023年4月18日,2023年4月19日
1,A,Q,46.918247,51.223557
7,D,A,53.999580,54.646510
17,G,V,541.108460,567.031300
31,K,L,92.850204,91.179840
36,L,K,69.283560,82.201220
45,M,U,157.934540,171.365100
58,Q,V,33.857630,40.268818
77,V,G,446.769840,518.284700


In [20]:
total = {
    '2023年4月18日总量': df_result['2023年4月18日'].sum(),
    '2023年4月19日总量': df_result['2023年4月19日'].sum()
}

In [21]:
total

{'2023年4月18日总量': 12154.6265816, '2023年4月19日总量': 11745.758671}